### Initialize lib

In [111]:
import sqlite3
import pandas as pd
import sqlalchemy as sa
import numpy as np

### Read all table list

In [43]:
pathfile = "../project/olist.db"

In [53]:
def readtable_list(path: str):
    conn = sqlite3.connect(path)
    df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    conn.close()
    return df['name'].tolist()


In [54]:
table_list = readtable_list(path=pathfile)

In [55]:
table_list

['olist_order_customer_dataset',
 'olist_order_dataset',
 'olist_order_reviews_dataset',
 'olist_order_payments_dataset',
 'olist_order_items_dataset',
 'olist_products_dataset',
 'olist_sellers_dataset',
 'olist_geolocation_dataset',
 'product_category_name_translation']

### Read table data and save into dataframe

In [65]:
def readtable_data(path: str, tablename : str):
    conn = sqlite3.connect(path)
    df = pd.read_sql_query(f"SELECT * FROM {tablename};", conn)
    conn.close()
    return df

In [67]:
df_customer = readtable_data(pathfile, 'olist_order_customer_dataset')
df_order = readtable_data(pathfile, 'olist_order_dataset')
df_review = readtable_data(pathfile, 'olist_order_reviews_dataset')
df_payment = readtable_data(pathfile, 'olist_order_payments_dataset')
df_order_item = readtable_data(pathfile, 'olist_order_items_dataset')
df_product = readtable_data(pathfile, 'olist_products_dataset')
df_seller = readtable_data(pathfile, 'olist_sellers_dataset')
df_geolocation = readtable_data(pathfile, 'olist_geolocation_dataset')
df_category_translation = readtable_data(pathfile, 'product_category_name_translation')

### Read info table

In [83]:
df_all = ['df_customer',
        'df_order',
        'df_review',
        'df_payment',
        'df_order_item',
        'df_product',
        'df_seller',
        'df_geolocation',
        'df_category_translation']

In [130]:
def readtable_info(table_name: list):
    all_df_info = []

    for df_name in table_name:
        df = eval(df_name)
        df_info = (pd.DataFrame({'Table Name': [df_name] * len(df.columns),
                            'Column': df.columns, 
                            'Data Type': [str(df[col].dtype) for col in df.columns], 
                            'Non-Null Count': [df[col].notnull().sum() for col in df.columns], 
                            'Total Length': [len(df[col]) for col in df.columns], 
                            'Null Count': [df[col].isnull().sum() for col in df.columns],
                            '% Null': [round(100 * df[col].isnull().mean(),2) for col in df.columns],
                            'Unique Count': [df[col].nunique() for col in df.columns]
                            })
                        )
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        if len(numeric_columns) > 0:
            for col in numeric_columns:
                desc = df[col].describe().round(1)
                df_info.loc[df_info['Column'] == col, 'mean'] = desc['mean']
                df_info.loc[df_info['Column'] == col, 'std'] = desc['std']
                df_info.loc[df_info['Column'] == col, 'min'] = desc['min']
                df_info.loc[df_info['Column'] == col, '25%'] = desc['25%']
                df_info.loc[df_info['Column'] == col, '50%'] = desc['50%']
                df_info.loc[df_info['Column'] == col, '75%'] = desc['75%']
                df_info.loc[df_info['Column'] == col, 'max'] = desc['max']

        all_df_info.append(df_info)

    return pd.concat(all_df_info)

In [142]:
table_info = readtable_info(df_all)

In [173]:
table_info.head(10)

,Table Name,Column,Data Type,Non-Null Count,Total Length,Null Count,% Null,Unique Count,mean,std,min,25%,50%,75%,max
0,df_customer,index,int64,99441,99441,0,0.0,99441,49720.0,28706.3,0.0,24860.0,49720.0,74580.0,99440.0
1,df_customer,customer_id,object,99441,99441,0,0.0,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,df_customer,customer_unique_id,object,99441,99441,0,0.0,96096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,df_customer,customer_zip_code_prefix,int64,99441,99441,0,0.0,14994,35137.5,29797.9,1003.0,11347.0,24416.0,58900.0,99990.0
4,df_customer,customer_city,object,99441,99441,0,0.0,4119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,df_customer,customer_state,object,99441,99441,0,0.0,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,df_order,index,int64,99441,99441,0,0.0,99441,49720.0,28706.3,0.0,24860.0,49720.0,74580.0,99440.0
1,df_order,order_id,object,99441,99441,0,0.0,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,df_order,customer_id,object,99441,99441,0,0.0,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,df_order,order_status,object,99441,99441,0,0.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Change object column into datetime column

In [169]:
table_column_dt = table_info.query("Column.str.contains('date') or \
                                    Column.str.contains('time') or \
                                    Column.str.contains('_at')", engine='python')[['Table Name', 'Column', 'Data Type']]

In [170]:
table_column_dt

,Table Name,Column,Data Type
4,df_order,order_purchase_timestamp,object
5,df_order,order_approved_at,object
6,df_order,order_delivered_carrier_date,object
7,df_order,order_delivered_customer_date,object
8,df_order,order_estimated_delivery_date,object
6,df_review,review_creation_date,object
7,df_review,review_answer_timestamp,object
5,df_order_item,shipping_limit_date,object


In [153]:
# change the object column into datetime
for i, row in table_column_dt.iterrows():
    df_name = row['Table Name']
    col_name = row['Column']
    eval(df_name)[col_name] = pd.to_datetime(eval(df_name)[col_name])
    eval(df_name)[col_name] = eval(df_name)[col_name].astype('datetime64[ns]')


In [171]:
readtable_info(df_all).query("Column.str.contains('date') or \
                            Column.str.contains('time') or \
                            Column.str.contains('_at')", engine='python')[['Table Name', 'Column', 'Data Type']]

,Table Name,Column,Data Type
4,df_order,order_purchase_timestamp,datetime64[ns]
5,df_order,order_approved_at,datetime64[ns]
6,df_order,order_delivered_carrier_date,datetime64[ns]
7,df_order,order_delivered_customer_date,datetime64[ns]
8,df_order,order_estimated_delivery_date,datetime64[ns]
6,df_review,review_creation_date,datetime64[ns]
7,df_review,review_answer_timestamp,datetime64[ns]
5,df_order_item,shipping_limit_date,datetime64[ns]


### Remove index column

In [160]:
table_info[table_info['Column'] == 'index']

,Table Name,Column,Data Type,Non-Null Count,Total Length,Null Count,% Null,Unique Count,mean,std,min,25%,50%,75%,max
0,df_customer,index,int64,99441,99441,0,0.0,99441,49720.0,28706.3,0.0,24860.0,49720.0,74580.0,99440.0
0,df_order,index,int64,99441,99441,0,0.0,99441,49720.0,28706.3,0.0,24860.0,49720.0,74580.0,99440.0
0,df_review,index,int64,99224,99224,0,0.0,99224,49611.5,28643.6,0.0,24805.8,49611.5,74417.2,99223.0
0,df_payment,index,int64,103886,103886,0,0.0,103886,51942.5,29989.4,0.0,25971.2,51942.5,77913.8,103885.0
0,df_order_item,index,int64,112650,112650,0,0.0,112650,56324.5,32519.4,0.0,28162.2,56324.5,84486.8,112649.0
0,df_product,index,int64,32951,32951,0,0.0,32951,16475.0,9512.3,0.0,8237.5,16475.0,24712.5,32950.0
0,df_seller,index,int64,3095,3095,0,0.0,3095,1547.0,893.6,0.0,773.5,1547.0,2320.5,3094.0
0,df_geolocation,index,int64,1000163,1000163,0,0.0,1000163,500081.0,288722.3,0.0,250040.5,500081.0,750121.5,1000162.0
0,df_category_translation,index,int64,71,71,0,0.0,71,35.0,20.6,0.0,17.5,35.0,52.5,70.0


In [ ]:
df_customer = readtable_data(pathfile, 'olist_order_customer_dataset')
df_order = readtable_data(pathfile, 'olist_order_dataset')
df_review = readtable_data(pathfile, 'olist_order_reviews_dataset')
df_payment = readtable_data(pathfile, 'olist_order_payments_dataset')
df_order_item = readtable_data(pathfile, 'olist_order_items_dataset')
df_product = readtable_data(pathfile, 'olist_products_dataset')
df_seller = readtable_data(pathfile, 'olist_sellers_dataset')
df_geolocation = readtable_data(pathfile, 'olist_geolocation_dataset')
df_category_translation = readtable_data(pathfile, 'product_category_name_translation')

In [174]:
dataframes = [df_customer, df_order, df_review, df_payment, df_order_item, df_product, df_seller, df_geolocation, df_category_translation]

for df in dataframes:
    df.drop(df.columns[0], axis=1, inplace=True)

In [176]:
readtable_info(['df_customer'])

,Table Name,Column,Data Type,Non-Null Count,Total Length,Null Count,% Null,Unique Count,mean,std,min,25%,50%,75%,max
0,df_customer,customer_id,object,99441,99441,0,0.0,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,df_customer,customer_unique_id,object,99441,99441,0,0.0,96096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,df_customer,customer_zip_code_prefix,int64,99441,99441,0,0.0,14994,35137.5,29797.9,1003.0,11347.0,24416.0,58900.0,99990.0
3,df_customer,customer_city,object,99441,99441,0,0.0,4119,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,df_customer,customer_state,object,99441,99441,0,0.0,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check Null Data

In [181]:
table_info = readtable_info(df_all)

In [185]:
table_info[table_info['Null Count'] > 0.0]

,Table Name,Column,Data Type,Non-Null Count,Total Length,Null Count,% Null,Unique Count,mean,std,min,25%,50%,75%,max
4,df_order,order_approved_at,datetime64[ns],99281,99441,160,0.16,90733,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,df_order,order_delivered_carrier_date,datetime64[ns],97658,99441,1783,1.79,81018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,df_order,order_delivered_customer_date,datetime64[ns],96476,99441,2965,2.98,95664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,df_review,review_comment_title,object,11568,99224,87656,88.34,4527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,df_review,review_comment_message,object,40977,99224,58247,58.70,36159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,df_product,product_category_name,object,32341,32951,610,1.85,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,df_product,product_name_lenght,float64,32341,32951,610,1.85,66,48.5,10.2,5.0,42.0,51.0,57.0,76.0
3,df_product,product_description_lenght,float64,32341,32951,610,1.85,2960,771.5,635.1,4.0,339.0,595.0,972.0,3992.0
4,df_product,product_photos_qty,float64,32341,32951,610,1.85,19,2.2,1.7,1.0,1.0,1.0,3.0,20.0
5,df_product,product_weight_g,float64,32949,32951,2,0.01,2204,2276.5,4282.0,0.0,300.0,700.0,1900.0,40425.0


In [187]:
readtable_info(['df_order', 'df_customer'])

,Table Name,Column,Data Type,Non-Null Count,Total Length,Null Count,% Null,Unique Count,mean,std,min,25%,50%,75%,max
0,df_order,order_id,object,99441,99441,0,0.00,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,df_order,customer_id,object,99441,99441,0,0.00,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,df_order,order_status,object,99441,99441,0,0.00,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,df_order,order_purchase_timestamp,datetime64[ns],99441,99441,0,0.00,98875,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,df_order,order_approved_at,datetime64[ns],99281,99441,160,0.16,90733,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,df_order,order_delivered_carrier_date,datetime64[ns],97658,99441,1783,1.79,81018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,df_order,order_delivered_customer_date,datetime64[ns],96476,99441,2965,2.98,95664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,df_order,order_estimated_delivery_date,datetime64[ns],99441,99441,0,0.00,459,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,df_customer,customer_id,object,99441,99441,0,0.00,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,df_customer,customer_unique_id,object,99441,99441,0,0.00,96096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
